# Pyloop

### Wojciech Łuszczyński
### Python 4 Begginers 2021
### Obsługa wyjątków w Pythonie, wirtualne środowiska, pliki

### Daft Academy Python4Beginners 30.11.2021

# 1 Wirtualne środowiska

## 1.1 Jak i po co zarządzać instalacją Pythona?

### 1.1.1 Wstęp - gdzie znajdziemy Pythona?

Domyślna instalacja Pythona na Linuksie. Wykonywalny program interpretera znajdziemy w katalogu z binarkami `/bin/`.

```console
$ which python
/bin/python
```

Biblioteki i reszta plików w tym i nasze pakiety znajdziemy w  `/usr/lib/`.

```console
$ ls /usr/lib/python3.7
-rw-r--r--  1 root root 5,5K 10-22 12:41 abc.py
-rwxr-xr-x  1 root root  20K 10-22 12:41 base64.py
...
drwxr-xr-x 88 root root 4,0K 11-16 15:26 site-packages
...
```

* `site-packages` to katalog, do którego trafiają zewnętrzne pakiety Pythona

### 1.1.2 `pip` Instalacja pakietów

`pip` - narzędzie dystrybuowane razem z Pythonem (od wersji 3.4)

`$ pip install [nazwa pakietu]`  / można też: `$ python -m pip install [nazwa pakietu]`

np. `pip install requests` 

Zainstaluje pakiet oraz jego zależności.

PIP instaluje pakiety do `site-packages` użytkownika (nie wymaga uprawnień administratora):

`$ pip install --user [nazwa paczki]`

### 1.1.3 Odinstalowanie pakietu

`$ pip uninstall [nazwa pakietu]`

Odinstaluje pakiet, pozostawi jego zależności. (jeśli takowe posiadał)

### 1.1.4 Lista zainstalowanych pakietów

```console
$ pip freeze
dateparser==0.7.0
humanize==0.5.1
pendulum==1.5.1
python-dateutil==2.7.5
[...]
```

Dzięki `$ pip freeze` Łatwo zapisać wszystkie pakiety z aktualnymi wersjami do pliku

`pip freeze > requirements.txt`

### 1.1.5 Instalacja z requirements.txt

`$ pip install -r requirements.txt`

### 1.1.6 Wyszukiwanie pakietów w (domyślnie w PyPI https://pypi.org/)

`$ pip search [nazwa pakietu]`

### 1.1.7 Inne cechy

* `pip` zapisuje pobrane pakiety w cache'u, dla szybszej ponownej instalacji
* `pip check` - sprawdzi, czy w środowisku zainstalowane są wszystkie wymagane zależności, oraz czy nie ma konfliktów (np. pakiet A wymaga pakietu C w wersji <1.0, zaś pakiet B wymaga pakietu C w wersji >1.0)
* Dokumentacja `pip` do poczytania: https://pip.pypa.io/en/stable/user_guide/

## 1.2 Czasami łatwo natrafić na problem(y)...

* W środowisku Pythona możemy mieć tylko jedną wersję danego pakietu, co może rodzić konflikty zależności:
    * np. pracujemy nad swoim projektem wykorzystującym najnowszą wersję pakietu A
    * i jednocześnie chcemy uruchomić projekt, w którym wymagana jest starsza wersja pakietu A.
* Na Linuksie i OSX Python jest używany przez system operacyjny. Pakiety (lub ich wersje), które doinstalujemy w głównym katalogu mogą zdestabilizować narzędzia systemowe...
* ...i odwrotnie, przy aktualizacji system może nam nadpisać wersje pakietów
* Możemy mieć problem z uporządkowaniem wymagań naszego projektu -- a chcielibyśmy móc uruchomić go nie tylko na naszym komputerze.

## 1.3 Środowisko wirtualne

### 1.3.1 Moduł `venv`

```console
$ python -m venv [nazwa_enva]
```

* Kopia środowiska Pythona, tworzona na żądanie
   * binarka `python`
   * biblioteka standardowa
   * `pip` + skrypty do aktywacji i dezaktywacji środowiska
   * instalujemy tylko pakiety, których potrzebujemy w konkretnym projekcie

* Izolacja od pozostałych środowisk 👍

### 1.3.2 Sposób użycia - moduł `venv`

```console
$ python -m venv some/path/to/some_project_venv
```

Utworzy środowisko w zadanym katalogu.

```console
$ ls some_project_venv
bin  include  lib  lib64
```

```console
$ ls -l some_project_venv/bin
-rw-r--r-- 1 daftcode daftcode 2248 11-23 11:15 activate
-rwxr-xr-x 1 daftcode daftcode  257 11-23 11:15 pip
lrwxrwxrwx 1 daftcode daftcode   11 11-23 11:15 python -> /bin/python
[...]
```

```console
$ ls some_project_venv/lib/Python3.7
site-packages
```

### 1.3.3. Aktywacja środowiska - skrypt shellowy `activate`

Platform  	|Shell 	|Command to activate virtual environment
---------------|----|-----------------------------------------
Posix 	|bash/zsh 	|`$ source <venv>/bin/activate`
  |  	fish 	|`$ . <venv>/bin/activate.fish`
  |  	csh/tcsh 	|`$ source <venv>/bin/activate.csh`
Windows 	|cmd.exe 	|`C:\> <venv>\Scripts\activate.bat`
  |  	PowerShell 	|`PS C:\> <venv>\Scripts\Activate.ps1`

https://docs.python.org/3/library/venv.html#creating-virtual-environments

### 1.3.4 Jak to działa?

skrypt `activate` wprowadza zmiany w bieżącej sesji terminala:

* modyfikacja *zmiennej środowiskowej* `PATH` - Skierowanie na `pythona`, `pip`a i inne pliki wykonywalne pod nowym adresem
* ustawienie zmiennej środowiskowej `VIRTUAL_ENV` - po uruchomieniu Python wie, skąd ładować importowane pakiety, a pip wie, gdzie instalować
* stworzenie komendy (funkcji w shellu) `deactivate`

## 1.4 Jak pracować z virtualenv ?

### 1.4.1 Przerażająca codzienność

Środowisko wirtualne w katalogu projektu:

* w katalogu projektu (powiedzmy `my_project`) odpalam `python -m venv env_my_project`
* [jeśli używam `git`a, wpisuję katalog `env_my_project` do pliku `.gitignore`]
* aktywuję środowisko `source env_my_project/bin/activate` (za każdym razem, gdy w nowym terminalu uruchamiam pliki projektu)
* instaluję potrzebne pakiety (to tylko raz)
* gdy projekt działa, zapisuję jego wymagania w pliku requirements: `pip freeze > requirements.txt`. Przyda się, gdy będę budować środowisko od nowa (`pip install -r requirements.txt`)
* gdy kończę pracę nad projektem i nie chcę zamykać okna terminala, dezaktywuję środowisko poleceniem `deactivate`

### 1.4.2 Dobre praktyki

* NIE instalujemy pakietów w głównym środowisku Pythona. To się może źle skończyć.
* narzędzia, do których chcemy mieć dostęp w całym systemie (jako nasz użytkownik) instalujemy przez `pip install --user [paczka]`
* do każdego projektu w pythonie używamy osobnego virtualenva

### 1.4.3 Drobne problemy użytkowania virtualneva

* Trzeba ręcznie stworzyć i nazwać środowisko wirtualne
* Trzeba pamiętać, jak mój virtualenv się nazywa i gdzie w systemie plików się znajduje
* Aby aktywować środowisko muszę wpisywać długie polecenia
* Środowisko wirtualne ma masę niepotrzebnych pakietów i nie mamy pewności, które dokładnie pakiety mogę usunąć

## 1.5 Trochę prościej - `virtualenvwrapper`

* napisany w PERLu wrapper na virtualenv
* środowiska wirtualne w jednym miejscu (virtualevwrapper przechowuje wszystkie envy w swoim katalogu domyślnym)
    * Trzymanie venv-ów w osobnym katalogu ułatwia pracę z projektem, GITem oraz IDE (jeśli kożystamy)
* wygodne polecenia, np. `mkvirtualenv [nazwa]`, `workon [nazwa]`
* możliwość skonfigurowania auto-aktywacji środowiska po wejściu do katalogu projektu
* dokumentacja: https://virtualenvwrapper.readthedocs.io/en/latest/

## 1.6 `pipenv` - poziom wyżej ??

https://pipenv.readthedocs.io/en/latest/

Napisany przez ludzi odpowiedzialnych z jeden z najlepszych pakietów w świecie Pythona `requests`
Rekomendowany (przez nich) system zarządzania zależnościami w projektach pythonowych.

Używa `pip` i `venv` za nas, dodając do tego wiele mądrych automatyzacji.

### 1.6.1 `pipenv` - szybki start

https://pipenv.readthedocs.io/en/latest/install/#installing-pipenv
    
OSX: `brew install pipenv`

Win/Linux: `pip install --user pipenv`

### 1.6.2 Codzienność z `pipenv`

* przechodzimy w konsoli do katalogu projektu: `cd my_project`
* od razu coś instalujemy, np. bibliotekę `requests`: `pipenv install requests`
* `pipenv` automatycznie znajduje/tworzy środowisko wirtualne (w swoim centralnym katalogu)
* automatycznie tworzy w projekcie pliki `Pipfile` i `Pipfile.lock` (lepszy odpowiednik `requirements.txt`)

```console
➜  experiments$ mkdir my_project
➜  experiments$ cd my_project 
➜  my_project$ pipenv install requests
Creating a virtualenv for this project…
Pipfile: /home/asia/experiments/my_project/Pipfile
Using /usr/bin/python (3.7.1) to create virtualenv…
✔ Complete 
Using interpreter /bin/python
New python executable in /home/asia/venvs/my_project-F8Fo0uYg/bin/python
Installing setuptools, pip, wheel...done.
Virtualenv location: /home/asia/venvs/my_project-F8Fo0uYg
Creating a Pipfile for this project…
Installing requests…
Adding requests to Pipfile's [packages]…
✔ Installation Succeeded 
Pipfile.lock not found, creating…
Locking [dev-packages] dependencies…
Locking [packages] dependencies…
✔ Success! 
Updated Pipfile.lock (444a6d)!
Installing dependencies from Pipfile.lock (444a6d)…
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 5/5 — 00:00:01
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.

```

### 1.6.3 `pipenv` - aktywacja środowiska

* aktywujemy środowisko poleceniem `pipenv shell`, dezaktywujemy poleceniem `exit` (uruchamiamy/kończymy nowy shell)
* możemy też uruchomić swoje programy bez wcześniejszej aktywacji, np. `pipenv run python my_program.py`

### 1.6.4 `pipenv` - Pipfile + Pipfile.lock

* `Pipfile` - deklarowane zależności projektu (np. `requests`) - czytelny dla człowieka
* każda instalacja/deinstalacja aktualizuje `Pipfile`
* `Pipfile.lock` - wszystkie zależności (czyli np. także zależności biblioteki `requests`), stan całego środowiska - czytelny dla pipenva

### 1.6.5 Prosty `Pipfile`

```toml
[[source]]
name = "pypi"
url = "https://pypi.org/simple"
verify_ssl = true

[dev-packages]

[packages]
requests = "*"

[requires]
python_version = "3.8"
```

### 1.6.6 `pipenv` - Sprzątanie środowiska

`$ pipenv uninstall [paczka]` - usuwa pakiet, aktualizuje Pipfile


`$ pipenv clean` - usunie niepotrzebne zależności pakietów, których już nie ma w Pipfile

# 2. Wyjątki

## 2.1 Syntax Error

- błąd interpretera podczas parsowania skryptu
- błąd składniowy
- domyślnie pythonowy interpreter poda linijkę i miejsce wystąpienia błędu składniowego i przestanie dalej przetwarzać plik

https://docs.python.org/3.8/tutorial/errors.html#syntax-errors

## 2.2 Wyjątki (`Exceptions`)

In [1]:
from typing import TextIO

f: TextIO = open("test.txt", "w")

In [2]:
f.write("Linia pierwsza\n")

15

In [3]:
f.write("Linia druga\n")

12

In [4]:
f.write("Linia trzecia\n")

14

In [5]:
x = 1 / 0
f.close()

ZeroDivisionError: division by zero

__Wyjątki__:
- błędy w trakcie wykonywania programu (Runtime Error)
- zazwyczaj zwracają jakiś opis błędu
- można sobie z nimi radzić

- https://docs.python.org/3.8/tutorial/errors.html#exceptions
- https://jeffknupp.com/blog/2013/02/06/write-cleaner-python-use-exceptions/

## 2.3 "Łapanie" wyjątków

### 2.3.1 Try Except

In [6]:
try:
    my_int: int = int("4")
    print(my_int)
except:
    pass

4


In [8]:
try:
    my_int: int = int("4")
    print(my_int)
except ValueError:
    print('Złapałem value error!')

4


- Słowo kluczowe `try` rozpoczyna blok "Łapania wyjątku"
- Słowo kluczowe `except` jest obowiązkowe jeśli rozpoczęliśmy blok `try`
- Na początek wykonywany jest kod w bloku `try`
- Jeżeli nie ma błędu, kod z `try` jest wykonany w całości, a kod w `except` jest pomijany

In [9]:
try:
    my_int: int = int("nie int :(")
    print(my_int)
except ValueError:
    print("Złapałem value error!")

Złapałem value error!


- Jeżeli pojawi się błąd, wykonanie kodu w bloku `try` jest przerwane
- Jeżeli błąd jest łapanego przez nas typu, to wykonywany jest kod zapisany w bloku `except`

In [10]:
try:
    my_int: int = int("4")
    print(my_int)
    a: dict = {"b": "c"}
    print(a["d"])
except (ValueError, KeyError): 
    print("Dostalem ValueError lub KeyError")

4
Dostalem ValueError lub KeyError


- Można łapać więcej niż jeden wyjątek jednocześnie

In [12]:
try:
    my_int: int = int("4")
    print(my_int)
    a: dict = {"b": "c"}
    print(a["d"])
except ValueError:
    print("Złapałem ValueError")
except KeyError:
    print("Złapałem KeyError")

4
Złapałem KeyError


- Obsługa wyjątków może przyjąć tak dużo bloków `except` ile potrzebujemy

In [13]:
try:
    my_int: int = int("4")
    print(my_int)
    a: dict = {"b": "c"}
    print(a.append("my_int"))
except ValueError:
    print("Złapałem ValueError")
except KeyError:
    print("Złapałem KeyError")    

4


AttributeError: 'dict' object has no attribute 'append'

- Jeżeli wystąpi nieobsłużony błąd - wyjątek nie jest złapany, a program zakończy działanie na tym błędzie

In [14]:
def fun(my_int: int, my_dict: dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict['a']
    except:
        print("Złapałem dowolny błąd")

In [12]:
fun("xx", {"a": "b"})

xx {'a': 'b'}
Złapałem dowolny błąd


In [10]:
fun("4", {"c": "d"})

4 {'c': 'd'}
Złapałem dowolny błąd


- Możliwe jest łapanie dowolnego rodzaju błędu - używać z rozwagą - zazwyczaj chcemy łapać konkretny błąd.
- Można łatwo przegapić zwykły błąd programistyczny gdy staramy się obsługiwać wszystkie błędy na raz

### 2.3.2 Wbudowane wyjątki

- https://docs.python.org/3.8/library/exceptions.html#bltin-exceptions

- wszystkie klasy wyjątków dziedziczą pośrednio lub bezpośrednio z BaseException
- w większości przypadków instancje klasy wyjątków dziedziczą z Exception (sprawdź w dokumentacji)
- python posiada bardzo rozbudowaną hierarchie błędów
- warto zobaczyć: https://docs.python.org/3.8/library/exceptions.html#exception-hierarchy

## 2.4 "Rzucanie" wyjątkiem

In [12]:
raise ValueError("Poważny błąd!")

ValueError: Poważny błąd!

- Możemy rzucać zarówno instancję wyjątku, jak i klasę

In [13]:
raise ValueError

ValueError: 

In [15]:
def fun(my_int: int, my_dict: dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict["a"]
    except ValueError:
        print("Złapałem ValueError")
        raise

In [16]:
fun("xx", {"a": "b"})

xx {'a': 'b'}
Złapałem ValueError


ValueError: invalid literal for int() with base 10: 'xx'

In [17]:
def fun(my_int: int, my_dict: dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict["a"]
    except ValueError as e:
        print("Złapałem ValueError")
        raise e

In [17]:
fun("xx", {"a": "b"})

xx {'a': 'b'}
Złapałem ValueError


ValueError: invalid literal for int() with base 10: 'xx'

## 2.5 Własna definicja klasy wyjątku

### 2.5.1 Definiowanie własnych błędów

- błąd powinien dziedziczyć po Exception
- zazwyczaj klasy błędów są proste
- często, gdy planujemy stworzyć więcej niż jeden błąd, tworzy się wspólną nadklasę dla modułu

In [20]:
class IncorrectGuessError(Exception):
    def __init__(self, difference):
        self.difference = difference

In [21]:
class NumberTooSmall(IncorrectGuessError):
    pass

In [22]:
class NumberTooBig(IncorrectGuessError):
    pass

In [23]:
def guess_number(guess: int):
    number = 10
    if guess > number:
        raise NumberToBig(guess - number)
    elif guess < number:
        raise NumberTooSmall(number - guess)
    else:
        print('Brawo')

In [24]:
try:
    guess_number(4)
except NumberTooSmall as exc:
    print('Za malo o {}'.format(exc.difference))
except NumberTooBig:
    print('Za duzo o {}'.format(exc.difference))

Za malo o 6


Dzięki instrukcji `else` możemy wykonać jakiś kod TYLKO wtedy, gdy kod w bloku `try` wykona się bez błędu

In [25]:
def fun(my_int:int, my_dict: dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict["a"]
    except ValueError:
        print("Złapałem błąd")
    else:
        print("Bez błędu!")

In [26]:
fun("4", {"a": "b"})

4 {'a': 'b'}
Bez błędu!


In [27]:
fun("x", {"a": "b"})

x {'a': 'b'}
Złapałem błąd


In [28]:
fun("4", {"c": "d"})

4 {'c': 'd'}


KeyError: 'a'

Kod, który chcemy żeby wykonał się zawsze umieszczamy w bloku `finally`

In [29]:
def fun(my_int, my_dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict['a']
    except ValueError:
        print('Złapałem błąd')
    else:
        print('Bez błędu!')
    finally:
        print('*' * 80)

In [30]:
fun("4", {"a": "b"})

4 {'a': 'b'}
Bez błędu!
********************************************************************************


In [31]:
fun("x", {"a": "b"})

x {'a': 'b'}
Złapałem błąd
********************************************************************************


In [32]:
fun("4", {"c": "d"})

4 {'c': 'd'}
********************************************************************************


KeyError: 'a'

kod w bloku `finally` wykonuje się zawsze
- niezależnie od tego czy wyjątek był obsłużony czy nie
- wykona się nawet jeśli w `try`, `except`, `else` nastąpi `return` lub `break`
- jeśli wyjątek nie jest obsłużony `finally` po wykonaniu kodu w swoim bloku rzuci ponownie tym samym wyjątkiem

# 3. Pliki

In [34]:
from typing import TextIO

f: TextIO = open("test.txt", "w")
f.write("Linia pierwsza\n")
f.write("Linia druga\n")
f.write("Linia trzecia\n")
f.close()

## 3.1 Otwieranie pliku

- `open` przyjmuje relatywną lub absolutną ścieżkę do pliku który otwieramy
- `open` przyjmuje też obiekt typu PathLib
- reszta argumentów jest opcjonalna, często używa się jedynie argumentu `mode`
- `mode` określa w jakim trybie otwieramy plik
- dostępne tryby: 'r' (__read__), 'w' (__write__), 'a' (__append__), 'r+' (__read__+__write__)
- domyślny tryb przy otwieraniu pliku to 'r' (__read__)
- __write__ nadpisuje plik, append dodaje do końca
- mode określa dodatkowo tryb tekstowy ('t' - domyślnie) lub binarny ('b')
- zwraca obiekt pliku
- domyślne kodowanie tekstu w trybie tekstowym to UTF-8

In [35]:
f: TextIO = open("test.txt", "w")
f.write("Linia pierwsza\n")
f.write("Linia druga\n")
f.write("Linia trzecia\n")

14

## Przydatne linki

- https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files
- https://docs.python.org/3/library/functions.html#open
- https://docs.python.org/3/glossary.html#term-file-object


## 3.2 Zamykanie pliku

- metoda `close` dostępna jest jako atrybyt obiektu pliku
- CPython dzięki `Garbage collectorowi` z licznikiem referencji sam zadba o zamknięcie pliku
- mimo działania `GC` __ZAWSZE__ powinniśmy zadbać o to sami - to, że CPython zamknie plik to `ficzer` tej konkretnej implementacji interpretera, część innych implementacji nie zapewnia obsługi tej funkcjonalności 

In [38]:
f: TextIO = open("test.txt", "w")
f.write("Linia pierwsza\n")
f.write("Linia druga\n")
f.write("Linia trzecia\n")
x = 1 / 0
f.close()

ZeroDivisionError: division by zero

In [39]:
try:
    f: TextIO = open("test.txt", "w")
    f.write("Linia pierwsza\n")
    f.write("Linia druga\n")
    f.write("Linia trzecia\n")
    x = 1 / 0
finally:
    print("Zamykam plik")
    f.close()
    print("Plik zamkniety")

Zamykam plik
Plik zamkniety


ZeroDivisionError: division by zero

O zamknięcie pliku da się zadbać łatwiej

In [40]:
with open("test.txt", "w") as f:
    f: TextIO
    f.write("Linia pierwsza\n")
    f.write("Linia druga\n")
    f.write("Linia trzecia\n")
    x = 1 / 0

ZeroDivisionError: division by zero

## 3.3 Dlaczego nie można zostawić otwartego pliku?

- wpisane przez nas zmiany mogą nie zostać zflushowane na dysk (zależne od implementacji - flush zapewnia, że dane z bufora zapisują się na dysku)
- systemy operacyjne mają limit otwartych plików - możemy dostać wyjątek w przypadku otwarcia zbyt dużej liczby plików
- czasami niezamknięty plik uniemożliwia odmontowanie, wyjęcie jakiegoś nośnika danych

### 3.3.1 Przydatne linki

- https://stackoverflow.com/questions/3167494/how-often-does-python-flush-to-a-file
- https://askubuntu.com/questions/701491/is-closing-a-file-after-having-opened-it-with-open-required-in-python

# 4 Context Manager

## 4.1 contextmanager

- obiekt, który definiuje w jakim kontekście ma się wywołać owijany kod
- określa, co ma się wykonać przed i po wykonaniu bloku kodu owijanego
- zazwyczaj wywoływane ze pomocą `with`, ale da się też bezpośrednio wywołać poszczególne metody
- przykłady użycia: zamykanie otwartych plików, zwalnianie zasobów/locków, przywracanie jakiegoś globalnego stanu

- https://docs.python.org/3/reference/datamodel.html#context-managers
- https://docs.python.org/3/library/stdtypes.html#typecontextmanager

## 4.1.1 with

- `with` jest używane do owinięcia bloku kodu za pomocą metod określonych w context managerze
- używane do zarządzania zasobami w danym bloku
- da się dzięki temu ładnie owinąć `try`...`except`...`finally`

- https://docs.python.org/3/reference/compound_stmts.html#with
- https://www.python.org/dev/peps/pep-0343/

In [96]:
with open('test.txt', 'w') as f:
    f: TextIO
    f.write('Linia pierwsza\n')
    f.write('Linia druga\n')
f.write('Linia trzecia\n')

ValueError: I/O operation on closed file.